<a href="https://colab.research.google.com/github/Lisker2/CS309_Project/blob/main/gene_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/nadavbra/shared_utils.git
!pip install protein-bert

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from sklearn.model_selection import train_test_split
from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs
from proteinbert import load_pretrained_model

In [ ]:
pretrained_model_generator, input_encoder = load_pretrained_model("/content/")

In [ ]:
train = pd.read_csv('protein_benchmarks/stability.train.csv')[:2000]
valid = pd.read_csv('protein_benchmarks/stability.valid.csv')[:300]
test = pd.read_csv('protein_benchmarks/stability.test.csv')[:2000]

train_X = train['seq']
valid_X = valid['seq']
test_X = test['seq']
train_Y = train['label']
valid_Y = valid['label']
test_Y = test['label']
print(train)
sequence_length = max(train.seq.map(lambda x: len(x)).max(), 
             test.seq.map(lambda x: len(x)).max(), 
             valid.seq.map(lambda x: len(x)).max()) + 2
print('sequence_length', sequence_length)

train_X = input_encoder.encode_X(train_X, sequence_length)
valid_X = input_encoder.encode_X(valid_X, sequence_length)
test_X = input_encoder.encode_X(test_X, sequence_length)

In [ ]:
model_generator = get_model_with_hidden_layers_as_outputs(pretrained_model_generator.create_model(sequence_length))

In [ ]:
local_representations_train, global_representations_train= model_generator.predict(train_X, batch_size = 64)
local_representations_test, global_representations_test= model_generator.predict(test_X, batch_size = 64)


In [ ]:
print(global_representations_train)

In [ ]:
model_stability = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape = local_representations_train[0].shape),
    tf.keras.layers.Dense(1, activation = 'softmax'),
]
)
model_stability.compile(loss='mse',
                  optimizer='adam',
                  metrics=['mse'])

training_callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(patience = 30, factor = 0.1, min_lr = 0.0001, verbose = 1),
    tf.keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True),
]
history_stability = model_stability.fit(local_representations_train, train_Y,
                                batch_size=64, epochs=30,
                                callbacks=training_callbacks)
     

In [ ]:
loss = history_stability.history['loss']
plt.plot(range(len(loss)), loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

In [ ]:
predict_Y = model_stability.predict(local_representations_test)
print(predict_Y)
p, _ = spearmanr(predict_Y, test_Y)
print(p)

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
non_cv = pd.read_csv('non.cv.txt')[::2]
non_cv.columns = ['seq']
print(len(non_cv))

label_1 = [0 for i in range(len(non_cv['seq']))]
non_cv['label'] = label_1

enhancer_cv = pd.read_csv('enhancer.cv.txt')[::2]

enhancer_cv.columns = ['seq']
print((len(enhancer_cv)))
label_1 = [1 for i in range(len(enhancer_cv))]
enhancer_cv['label'] = label_1

In [ ]:
enhancer_train = pd.concat([non_cv, enhancer_cv],axis=0)
enhancer_train.index = range(len(enhancer_train))
enhancer_train.head()

In [ ]:
non_ind = pd.read_csv('non.ind.txt')[::2]
non_ind.columns = ['seq']
print((len(non_ind)))

label_1 = [0 for i in range(len(non_ind))]
non_ind['label'] = label_1

enhancer_ind = pd.read_csv('enhancer.ind.txt')[::2]
enhancer_ind.columns = ['seq']
print((len(enhancer_ind)))

label_1 = [1 for i in range(len(enhancer_ind))]
enhancer_ind['label'] = label_1

# 新段落

In [ ]:
enhancer_test = pd.concat([non_ind, enhancer_ind],axis=0)
enhancer_test.index = range(len(enhancer_test))
enhancer_test.head()

In [ ]:
enhancer_valid=enhancer_train.sample(n=None, frac=0.1, replace=False, weights=None, random_state=None, axis=0)

In [ ]:
train_X = enhancer_train['seq']
test_X = enhancer_test['seq']
valid_X= enhancer_valid['seq']
train_Y = enhancer_train['label']
test_Y = enhancer_test['label']
valid_Y= enhancer_valid['label']
sequence_length = max(enhancer_train.seq.map(lambda x: len(x)).max(), 
             enhancer_test.seq.map(lambda x: len(x)).max(),
             enhancer_valid.seq.map(lambda x: len(x)).max())+2
print('sequence_length', sequence_length)

In [ ]:
train_X = input_encoder.encode_X(train_X, sequence_length)
test_X = input_encoder.encode_X(test_X, sequence_length)
valid_X= input_encoder.encode_X(valid_X, sequence_length)

In [ ]:
model_generator = get_model_with_hidden_layers_as_outputs(pretrained_model_generator.create_model(sequence_length))

In [ ]:
local_representations_train, global_representations_train= model_generator.predict(train_X, batch_size = 64)
local_representations_test, global_representations_test= model_generator.predict(test_X, batch_size = 64)
local_representations_valid, global_representations_valid= model_generator.predict(valid_X, batch_size = 64)

In [ ]:
model_stability = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, 3, activation='relu',input_shape=(202,1562,1)),
    tf.keras.layers.MaxPooling2D((2)),
    tf.keras.layers.Conv2D(64, 3, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D((2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation = 'sigmoid'),
    tf.keras.layers.Dense(64, activation = 'linear'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model_stability.compile(loss=tf.keras.losses.binary_crossentropy,
                  optimizer='adam',
                  metrics=['binary_accuracy'])

training_callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(patience = 5, factor = 0.2, min_lr = 0.001, verbose = 1),
    tf.keras.callbacks.EarlyStopping(monitor='binary_accuracy',patience = 3, restore_best_weights = True, verbose=1),
]

history_stability = model_stability.fit(local_representations_train, train_Y,
                                validation_data=(local_representations_valid, valid_Y),
                                batch_size=32, epochs=50,callbacks=training_callbacks)
     

In [ ]:
predict_Y = model_stability.predict(local_representations_test)
p, _ = spearmanr(predict_Y, test_Y)
print(p)

In [ ]:
loss = history_stability.history['binary_accuracy']
plt.plot(range(len(loss)), loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

In [ ]:
predict_Y=model_stability.predict(local_representations_test)

for i in range(len(predict_Y)):
  if(predict_Y[i]>0.5):
    predict_Y[i]=1
  else:
    predict_Y[i]=0
error=0
test_Y=np.array(test_Y)
for i in range(len(predict_Y)):
  error+=abs(predict_Y[i]-test_Y[i])
print(1-error/len(predict_Y))

In [ ]:
def calc(predict_Y,test_Y):
  for i in range(len(predict_Y)):
    if(predict_Y[i]>0.5):
      predict_Y[i]=1
    else:
      predict_Y[i]=0
  error=0
  test_Y=np.array(test_Y)
  for i in range(len(predict_Y)):
    error+=abs(predict_Y[i]-test_Y[i])
  return(1-error/len(predict_Y))


In [ ]:
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
cross_data=shuffle(enhancer_train).reset_index(drop=True)
cross_data_X=cross_data['seq']
cross_data_Y=cross_data['label']
kf=KFold(n_splits=6)
kf.get_n_splits(cross_data_X)
KFold(n_splits=6, random_state=None, shuffle=False)


     
for i, (train_index,test_index) in enumerate(kf.split(cross_data_X)):
  train_X=cross_data_X[train_index]
  test_X=cross_data_X[test_index]
  train_Y = cross_data_Y[train_index]
  test_Y=cross_data_Y[test_index]

  train_X = input_encoder.encode_X(train_X, sequence_length)
  test_X= input_encoder.encode_X(test_X, sequence_length)
  
  local_representations_train, global_representations_train= model_generator.predict(train_X, batch_size = 64)
  local_representations_test, global_representations_test= model_generator.predict(test_X, batch_size = 64)
  
  tf.keras.backend.clear_session()

  model_stability = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, 3, 3, activation='relu',input_shape=(202,1562,1)),
      tf.keras.layers.MaxPooling2D((2)),
      tf.keras.layers.Conv2D(64, 3, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D((2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation = 'sigmoid'),
      tf.keras.layers.Dense(1, activation = 'sigmoid')
  ])
  model_stability.compile(loss=tf.keras.losses.binary_crossentropy,
                  optimizer='adam',
                  metrics=['binary_accuracy'])
  
  history_stability = model_stability.fit(local_representations_train, train_Y,
                                batch_size=32, epochs=20)

  predict_Y=model_stability.predict(local_representations_test)
  print(calc(predict_Y,test_Y)) 

  if(i==0):
    model_stability.save("my_model")
  if(i==1):
    model_stability.save("my_mode2")
  if(i==2):
    model_stability.save("my_mode3")
  if(i==3):
    model_stability.save("my_mode4")
  if(i==4):
    model_stability.save("my_mode5")
  tf.keras.backend.clear_session()

loss = history_stability.history['binary_accuracy']
plt.plot(range(len(loss)), loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

In [ ]:
reconstructed_model = keras.models.load_model("my_mode2")
predict_Y=reconstructed_model.predict(local_representations_test)
print(test_Y)
print(len(predict_Y))
print(calc(predict_Y,test_Y)) 
tf.keras.backend.clear_session()